In [373]:
import kfp

from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

In [374]:
URL='http://your-load-balancer.elb.amazonaws.com'
AUTH="authservice_session=<your token>"

In [375]:
client = kfp.Client(host=URL+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace="admin")
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 2, 22, 10, 16, 13, tzinfo=tzlocal()),
 'description': None,
 'id': 'bc05a15c-5ac6-4cdf-935b-7a6451089d2c',
 'name': 'sample',
 'resource_references': [{'key': {'id': 'admin', 'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

In [376]:
# load components (note the components are not platform specific, but the importers are)
# data_prep_op = load_component_from_file(f"data_prep_step/{model}/component.yaml")
# train_model_op = load_component_from_file(f"training_step/{model}/component.yaml")
list_item_op = load_component_from_file("file/component.yaml")
download_op = load_component_from_file("web/component.yaml")
model_archive_op = load_component_from_file("model_archive_step/component.yaml")
deploy_op = load_component_from_url("https://raw.githubusercontent.com/kubeflow/pipelines/97eae83a96b0ac87805e2d6db6097e479bb38b1f/components/kubeflow/kfserving/component.yaml")

# Using dsl for model archiver and torchserve integration

In [412]:
@dsl.pipeline(
    name='pytorch',
    description='hello world'
)
def train_imagenet_pytorch(
    input_directory: str = "/pv/input",
    output_directory: str = "/pv/outputs",
    handlerFile: str = "image_classifier"
):
    vop = dsl.VolumeOp(
        name="newpvc",
        resource_name="newpvc",
        modes=dsl.VOLUME_MODE_RWO,
        size="1Gi"
    )

    @dsl.component
    def download(url: str, output_path:str):
        return dsl.ContainerOp(
            name='Download',
            image='busybox:latest',
            command=["sh", "-c"],
            arguments=["mkdir -p %s; wget %s -P %s" % (output_path, url, output_path)],
        )

    # Using mutiple downloads as wget cannot download folder from s3
    properties = download(url='https://kubeflow-dataset.s3.us-east-2.amazonaws.com/model_archive/properties.json', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(vop)
    model = download(url='https://raw.githubusercontent.com/pytorch/serve/master/examples/image_classifier/mnist/mnist.py', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(properties)
    serialized = download(url='https://raw.githubusercontent.com/pytorch/serve/master/examples/image_classifier/mnist/mnist_cnn.pt', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(model)
    handler = download(url='https://raw.githubusercontent.com/pytorch/serve/master/examples/image_classifier/mnist/mnist_handler.py', output_path="/pv/input").add_pvolumes({"/pv":vop.volume}).after(serialized)
           
    op_task = dsl.ContainerOp(
        name='main',
        image="quay.io/aipipeline/kfserving-component:v0.5.0",
        command=['python'],
        arguments=[
          "-u", "kfservingdeployer.py",
          "--action", "apply",
          "--model-name", "torchserve",
          "--model-uri", "pvc://{{workflow.name}}-newpvc/outputs",
          "--namespace", "admin",
          "--framework", "pytorch",
        ],
        pvolumes={"/pv": vop.volume},
        # pass in init_container list
        init_containers=[
            dsl.UserContainer(
                name='init',
                image='jagadeeshj/model_archive_step:kfpv1.0',
                command=["/usr/local/bin/dockerd-entrypoint.sh"],
                args=[
                    "--output_path", output_directory,
                    "--input_path", input_directory,
                    "--handlerfile", handlerFile
                ],
                mirror_volume_mounts=True,),
        ],
    ).after(handler)

#     deploy = deploy_op(
#         action="create", 
#         model_name="torchserve", 
#         model_uri="pvc://{{workflow.name}}-newpvc/outputs", 
#         namespace='admin',
#         framework='pytorch'
#     ).after(op_task)


In [413]:
# Compile pipeline
compiler.Compiler().compile(train_imagenet_pytorch, 'pytorch.tar.gz', type_check=True)

In [414]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-model', 'pytorch.tar.gz')

### Wait for inference service below to go to `READY True` state.

In [421]:
DEPLOY='torchserve'
MODEL='mnist'

In [422]:
!kubectl get isvc $DEPLOY

NAME         URL                                   READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                  AGE
torchserve   http://torchserve.admin.example.com   True           100                              torchserve-predictor-default-gvkhf   51s


# Inference

In [417]:
!kubectl get isvc $DEPLOY -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3

torchserve.admin.example.com


In [418]:
S_HOSTNAME=!kubectl get isvc $DEPLOY  -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3
SERVICE_HOSTNAME=S_HOSTNAME[0]
SERVICE_HOSTNAME

'torchserve.admin.example.com'

In [423]:
#!curl -v -H "Host: $SERVICE_HOSTNAME" -H "Cookie: $AUTH" "$URL/v1/models/$MODEL:predict" -d @./mnist.json

# Explain

In [424]:
#!curl -v -H "Host: $SERVICE_HOSTNAME" -H "Cookie: $AUTH" "$URL/v1/models/$MODEL:explain" -d @./mnist.json